In [1]:
import tensorflow as tf
import cv2
import os
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM,  Conv2DTranspose, Input, Conv2D, MaxPooling2D, UpSampling2D, \
TimeDistributed, ConvLSTM2D, Reshape, Lambda, Flatten, Concatenate, Conv3D, MaxPooling3D, UpSampling3D, Conv3DTranspose
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model

In [2]:
X=[]
y=[]

In [3]:
data_dir= "M:\\Exposure_dataset\\INPUT"
GT_dir= "M:\\Exposure_dataset\\GT"

In [4]:
for img in os.listdir(data_dir):
    image= cv2.imread(os.path.join(data_dir, img))
    image= cv2.resize(image, (64, 64))
    X.append(image)

In [5]:
for img in os.listdir(GT_dir):
    image= cv2.imread(os.path.join(GT_dir, img))
    image= cv2.resize(image, (64, 64))
    y.append(image)

In [6]:
X_data=[]
y_data=[]

In [7]:
len(X)

24330

In [8]:
len(y)

4866

In [9]:
sub_list=[]
for i in range(len(X)):
    sub_list.append(X[i])
    if (i%5==4):
        y_data.append(y[i//5])
        X_data.append(sub_list)
        sub_list=[]

In [10]:
X= []
y=[]

In [11]:
len(X_data)

4866

In [12]:
len(y_data)

4866

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [ ]:
X_train = np.array(X_train, dtype = 'float')/255
X_test = np.array(X_test, dtype = 'float')/255
X_val = np.array(X_val, dtype = 'float')/255

y_train = np.array(y_train, dtype = 'float')/255
y_test = np.array(y_test, dtype = 'float')/255
y_val = np.array(y_val, dtype = 'float')/255

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_val.shape

In [ ]:
clear_session()
tf.compat.v1.reset_default_graph()

In [26]:
batch_input_shape= (5, 128, 128, 3)

In [23]:
def encoder(inputs):
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    return conv1

In [27]:
batch_input = Input(shape=batch_input_shape)

lst=[]

for i in range(batch_input.shape[1]):
    inputt = Lambda(lambda x: x[:, i])(batch_input)
    lst.append(inputt)

for i in range(len(lst)):
    lst[i]= encoder(lst[i])

conv1= Concatenate()(lst)

pool1 = MaxPooling2D((2, 2))(conv1)  # Adjust pool size according to your data

conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Decoder (upsampling)
up1 = Conv2DTranspose(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(pool2)

concat1 = Concatenate(axis=-1)([conv2, up1])
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up2 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(conv3)

concat2 = Concatenate(axis=-1)([conv1, up2])

conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
output = Conv2D(3, (1,1), activation='sigmoid')(conv4)

In [18]:
model = Model(inputs=batch_input, outputs=output)

In [19]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 5, 128, 128, 0                                            
__________________________________________________________________________________________________
lambda_25 (Lambda)              (None, 128, 128, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
lambda_26 (Lambda)              (None, 128, 128, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
lambda_27 (Lambda)              (None, 128, 128, 3)  0           input_11[0][0]                   
______________________________________________________________________________________________

In [ ]:
with tf.device('/CPU:0'):
    history= model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=6, batch_size=8)


In [ ]:
model.save("my_model.h5")